# References

In [1]:
# http://theautomatic.net/yahoo_fin-documentation/
# http://theautomatic.net/2020/05/05/how-to-download-fundamentals-data-with-python/
# https://algotrading101.com/learn/yahoo-finance-api-guide/>
# https://blog.quantinsti.com/quantitative-value-investing-strategy-python/
# https://www.activestate.com/blog/top-10-python-packages-for-finance-and-financial-modeling/

# Imports

In [2]:
import pandas as pd
import yahoo_fin.stock_info as si
# import time

# Variables

In [3]:
ratio_valuation_function=['Price/Book (mrq)','Trailing P/E','Forward P/E 1',\
                          'PEG Ratio (5 yr expected) 1','Price/Sales (ttm)']

ratio_stat=['Total Debt/Equity (mrq)', 'Diluted EPS (ttm)', 'Trailing Annual Dividend Yield 3',\
            'Forward Annual Dividend Yield 4', '% Held by Insiders 1','% Held by Institutions 1',\
            'Return on Equity (ttm)','Return on Assets (ttm)','Quarterly Earnings Growth (yoy)',\
            'current_price']

# ratio_stat=['Total Debt/Equity (mrq)', 'Diluted EPS (ttm)', 'Trailing Annual Dividend Yield 3',\
#             'Forward Annual Dividend Yield 4', '% Held by Insiders 1','% Held by Institutions 1',\
#             'Return on Equity (ttm)','Return on Assets (ttm)','Quarterly Earnings Growth (yoy)', \
#             'current_price','Beta (5Y Monthly)']

# Get Updated S&P 500

In [4]:
# complete sp 500 list
# file generated: get_sp_500_update.py
# ticker_df = pd.read_csv('../ProjectDatasets/sp_500_symbols.csv', sep=',')
# tickers = ticker_df['Symbol'].tolist()

# get latest sp500
tickers = si.tickers_sp500()

# tickersDF = si.tickers_sp500(True)
# tickers = tickers[:4].copy()

print(len(tickers))

505


# Company Info

In [ ]:
tickersDF = si.tickers_sp500(True)
tickersDF.index=tickersDF['Symbol']

tickersDF.index.rename('ticker', inplace=True)
tickersDF.drop(columns=['Symbol'], inplace=True)

# Sort Data

In [5]:
table=pd.DataFrame()

ticker_index = []
retry_ticker = []
count = 0

for p in tickers:
#     print(p)
    try:
        data=si.get_stats(p)
        data.index=data["Attribute"]
        data=data.drop(labels="Attribute",axis=1)
        raw_table=data.T
        raw_table['current_price'] = round(si.get_live_price(p),2)
        table=table.append(raw_table)   #Table having Data about the company
        ticker_index.append(p)
    except:
        count = count+1
        print('Bad Ticker {}: {}'.format(count, p))
        retry_ticker.append(p)

if len(retry_ticker) > 0:     
    time.sleep(60*20)
    count = 0

    for p in retry_ticker:
    #     print(p)
        try:
            data=si.get_stats(p)
            data.index=data["Attribute"]
            data=data.drop(labels="Attribute",axis=1)
            raw_table=data.T
            raw_table['current_price'] = round(si.get_live_price(p),2)
            table=table.append(raw_table)   #Table having Data about the company
            ticker_index.append(p)
            time.sleep(30)
        except:
            count = count+1
            print('Bad Ticker 2nd attempt {}: {}'.format(count, p))
            time.sleep(60*5)
    
table.index=ticker_index
table1 = table[ratio_stat]

Bad Ticker 1: BRK.B
Bad Ticker 2: BF.B
Bad Ticker 2nd attempt 1: BRK.B
Bad Ticker 2nd attempt 2: BF.B


In [6]:
time.sleep(60*30)

In [7]:
table=pd.DataFrame()

tickers = ticker_index.copy()
new_index = []
retry_ticker = []
count = 0

for p in tickers:
#     print(p)
    try:
        extra_ratio=si.get_stats_valuation(p)
        extra_ratio = extra_ratio.iloc[:,0:2]
        extra_ratio.index=extra_ratio['Unnamed: 0']
        extra_ratio=extra_ratio.drop(labels='Unnamed: 0',axis=1)
        new_table=extra_ratio.T
        table=table.append(new_table)  #Table having Data about the company
        new_index.append(p)
        time.sleep(30)
    except:
        count = count+1
        print('Bad Ticker {}: {}'.format(count, p))
        retry_ticker.append(p)
        time.sleep(60*20)
        
if len(retry_ticker) > 0:     
    time.sleep(60*20)
    count = 0

    for p in retry_ticker:
    #     print(p)
        try:
            data=si.get_stats(p)
            data.index=data["Attribute"]
            data=data.drop(labels="Attribute",axis=1)
            raw_table=data.T
            raw_table['current_price'] = round(si.get_live_price(p),2)
            table=table.append(raw_table)   #Table having Data about the company
            new_index.append(p)
            time.sleep(30)
        except:
            count = count+1
            print('Bad Ticker 2nd attempt {}: {}'.format(count, p))
            time.sleep(60*5)
        
table.index=new_index
table2 = table[ratio_valuation_function]

In [8]:
final=pd.concat([table2,table1],axis=1)
# og_final = final.copy()

In [10]:
final.to_csv('../ProjectDatasets/final_recommendations_int.csv', index=True, index_label='ticker')
print(final.shape)

(503, 15)


In [11]:
# final = pd.read_csv('../ProjectDatasets/final_recommendations_int.csv', sep=',', index_col='ticker')

# Evaluations

In [12]:
final['Trailing P/E'] = pd.to_numeric(final['Trailing P/E'], errors='coerce')
final['Price/Book (mrq)'] = pd.to_numeric(final['Price/Book (mrq)'], errors='coerce')

In [13]:
# low_valuations
final = final[(final['Trailing P/E'].astype(float)<40) & (final['Price/Book (mrq)'].astype(float) < 15)].copy()

# earning_power
final = final[final['Diluted EPS (ttm)'].astype(float) > 4].copy()

# equity_to_debt
final = final[(final['Total Debt/Equity (mrq)'].astype(float)< 75 )].copy() # Filter for Debt to Equity
final = final[(final['Return on Equity (ttm)'] > str(20) )] # Filter for ROE

# insider_owned
final = final[final['% Held by Insiders 1']>str(.07)]

In [ ]:
FINAL = pd.concat([tickersDF,final], axis=1, join='inner')

In [14]:
FINAL.head()

,Price/Book (mrq),Trailing P/E,Forward P/E 1,PEG Ratio (5 yr expected) 1,Price/Sales (ttm),Total Debt/Equity (mrq),Diluted EPS (ttm),Trailing Annual Dividend Yield 3,Forward Annual Dividend Yield 4,% Held by Insiders 1,% Held by Institutions 1,Return on Equity (ttm),Return on Assets (ttm),Quarterly Earnings Growth (yoy),current_price
ACN,10.13,34.57,34.48,3.69,4.13,18.44,8.45,1.18%,1.21%,0.11%,73.54%,31.88%,11.57%,16.70%,289.52
ARE,2.26,29.84,47.17,NaN,12.03,58.83,6.01,2.36%,2.43%,1.13%,92.76%,7.01%,1.64%,118.10%,179.57
AMAT,10.79,32.26,22.57,1.80,6.83,49.66,4.18,0.65%,0.71%,0.37%,82.74%,38.31%,14.14%,26.70%,137.30
APTV,4.96,21.76,38.17,1.59,3.00,55.57,6.66,0.15%,NaN,0.55%,97.73%,30.09%,3.27%,30.00%,145.20
ANET,7.32,39.88,31.65,2.27,10.93,2.72,7.99,NaN,NaN,27.91%,65.04%,20.42%,9.89%,-29.80%,321.07


In [21]:
FINAL.to_csv('../ProjectDatasets/final_recommendations.csv', index=True, index_label='ticker')

In [22]:
FINAL.shape

(38, 15)